<a href="https://colab.research.google.com/github/louiscollinsjr/componentSnippets/blob/main/ServeSync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00


In [ ]:
"""ServeSync - Seamlessly create schedules with constraints for volunteers."""
from typing import Union
from ortools.sat.python import cp_model

from dataclasses import dataclass   # Import dataclass for data classes
from faker import Faker # Import Faker for generating random data
import random

In [ ]:
# This program tries to find an optimal assignment of volunteer group to shifts
# (5 shifts per day, for 7 days), subject to some constraints (see below).
# Each volunteer group can request to be assigned to specific shifts.
# The optimal assignment maximizes the number of fulfilled shift requests.

# Constants
NUM_VOLUNTEERS = 215
NUM_SHIFTS = 5  # Number of shifts per day
NUM_WEEKS = 4  # 12 weeks (3 months)
SCHEDULING_HORIZON = NUM_WEEKS * 7
VOLUNTEERS_PER_SHIFT = 3  # New constant for volunteers per shift

all_volunteers = range(NUM_VOLUNTEERS)
all_shifts = range(NUM_SHIFTS)
all_days = range(SCHEDULING_HORIZON)

@dataclass
class Volunteer:
    id: int
    name: str
    email: str
    phone: str
    address: str
    male: bool
    automobile: bool
    pioneer_status: bool
    weekly_availablity: list[list[int]]

fake = Faker()

# Create a list of Volunteer objects
volunteers = [
    Volunteer(
        id=fake.unique.random_number(digits=8),
        name=fake.name(),
        email=fake.unique.email(),
        address=fake.address(),
        phone=f"+04-{i:03d}-{random.randint(1000, 9999)}",
        male=random.choice([True, False]),
        automobile=random.choice([True, False]),
        pioneer_status=random.choice([True, False]),
        # Generate random weekly availablity
         weekly_availablity=[
            [random.randint(0, 1) for _ in range(NUM_SHIFTS)] for _ in range(7)  # 7 days, 5 shifts each
        ]
    )
    for i in range(NUM_VOLUNTEERS)
]

#print (volunteers)

In [ ]:
# Generate random availability for demonstration purposes
# shift_availability = [
#     [[random.randint(0, 1) for _ in range(NUM_SHIFTS)] for _ in range(SCHEDULING_HORIZON)]
#     for _ in range(NUM_VOLUNTEERS)
# ]

shift_availability = {}

# Populate shift_availability using volunteer data and ID
for volunteer in volunteers:
    # Repeat weekly_availability for the entire scheduling horizon (4 weeks)
    volunteer_availability = volunteer.weekly_availablity * NUM_WEEKS
    shift_availability[volunteer.id] = volunteer_availability

# print(shift_availability)
# Print only the first 5 records
print("First 5 records of shift_availability:")
count = 0
for volunteer_id, availability in shift_availability.items():
    print(f"Volunteer ID: {volunteer_id}, Availability: {availability}")
    count += 1
    if count == 5:
        break

First 5 records of shift_availability:
Volunteer ID: 42166616, Availability: [[0, 0, 1, 1, 1], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 1, 1], [0, 0, 1, 1, 0]]
Volunteer ID: 25621424, Availability: [[0, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1], [0, 0, 0, 1, 0], [0, 1, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 1, 1, 1],

In [ ]:
# Create the model.
model = cp_model.CpModel()

In [ ]:
# Create shift variables
# shifts = {}
# for n in all_volunteers:
#     for d in all_days:
#         for s in all_shifts:
#             shifts[(n, d, s)] = model.NewBoolVar(f"shift_volunteer_num{n}_day{d}_shift{s}")

shifts = {}
for volunteer_id, availability in shift_availability.items():
    for d in all_days:
        for s in all_shifts:
            # Access availability for this volunteer, day, and shift
            is_available = availability[d][s]
            shifts[(volunteer_id, d, s)] = model.NewBoolVar(f"shift_v{volunteer_id}_d{d}_s{s}")

print(shifts)

{(42166616, 0, 0): shift_v42166616_d0_s0(0..1), (42166616, 0, 1): shift_v42166616_d0_s1(0..1), (42166616, 0, 2): shift_v42166616_d0_s2(0..1), (42166616, 0, 3): shift_v42166616_d0_s3(0..1), (42166616, 0, 4): shift_v42166616_d0_s4(0..1), (42166616, 1, 0): shift_v42166616_d1_s0(0..1), (42166616, 1, 1): shift_v42166616_d1_s1(0..1), (42166616, 1, 2): shift_v42166616_d1_s2(0..1), (42166616, 1, 3): shift_v42166616_d1_s3(0..1), (42166616, 1, 4): shift_v42166616_d1_s4(0..1), (42166616, 2, 0): shift_v42166616_d2_s0(0..1), (42166616, 2, 1): shift_v42166616_d2_s1(0..1), (42166616, 2, 2): shift_v42166616_d2_s2(0..1), (42166616, 2, 3): shift_v42166616_d2_s3(0..1), (42166616, 2, 4): shift_v42166616_d2_s4(0..1), (42166616, 3, 0): shift_v42166616_d3_s0(0..1), (42166616, 3, 1): shift_v42166616_d3_s1(0..1), (42166616, 3, 2): shift_v42166616_d3_s2(0..1), (42166616, 3, 3): shift_v42166616_d3_s3(0..1), (42166616, 3, 4): shift_v42166616_d3_s4(0..1), (42166616, 4, 0): shift_v42166616_d4_s0(0..1), (42166616, 4

In [ ]:
# Constraint 1: Each volunteer works at most one shift per 14 days
# for n in all_volunteers:
#     for start_day in range(0, SCHEDULING_HORIZON - 13):
#         model.Add(sum(shifts[(n, d, s)] for d in range(start_day, start_day + 14) for s in all_shifts) <= 1)

# # Constraint 2: Each shift each day is assigned to exactly three volunteers
# for d in all_days:
#     for s in all_shifts:
#         model.Add(sum(shifts[(n, d, s)] for n in all_volunteers) == VOLUNTEERS_PER_SHIFT)

# Constraint 1: Each volunteer works at most one shift per 14 days
for volunteer_id in shift_availability:  # Iterate through volunteer IDs in shift_availability
    for start_day in range(0, SCHEDULING_HORIZON - 13):
        model.Add(sum(shifts.get((volunteer_id, d, s), 0)  # Use get with default 0 for missing keys
                      for d in range(start_day, start_day + 14)
                      for s in all_shifts) <= 1)

# Constraint 2: Each shift each day is assigned to exactly three volunteers
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts.get((volunteer_id, d, s), 0)  # Use get with default 0
                      for volunteer_id in shift_availability) == VOLUNTEERS_PER_SHIFT)

# Constraint 3: Each shift must have at least one male volunteer
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts.get((volunteer_id, d, s), 0)
                      for volunteer_id in shift_availability
                      # Check if the volunteer is male using a list comprehension or filter
                      if any(v.id == volunteer_id and v.male == 'True' for v in volunteers)) >= 1)

In [ ]:
# Create variables to track the total shifts for each volunteer
# total_shifts = {}
# for n in all_volunteers:
#     total_shifts[n] = model.NewIntVar(0, SCHEDULING_HORIZON, f"total_shifts_volunteer_{n}")
#     model.Add(total_shifts[n] == sum(shifts[(n, d, s)] for d in all_days for s in all_shifts))

# # Create a variable for the maximum number of shifts assigned to any volunteer
# max_shifts = model.NewIntVar(0, SCHEDULING_HORIZON, "max_shifts")
# model.AddMaxEquality(max_shifts, [total_shifts[n] for n in all_volunteers])

# # Create a variable for the minimum number of shifts assigned to any volunteer
# min_shifts = model.NewIntVar(0, SCHEDULING_HORIZON, "min_shifts")
# model.AddMinEquality(min_shifts, [total_shifts[n] for n in all_volunteers])

# Create variables to track the total shifts for each volunteer
total_shifts = {}
# Iterate over the volunteer IDs present in shift_availability
for volunteer_id in shift_availability:
    total_shifts[volunteer_id] = model.NewIntVar(0, SCHEDULING_HORIZON, f"total_shifts_volunteer_{volunteer_id}")
    # Use get to access shifts, providing 0 as default for missing keys
    model.Add(total_shifts[volunteer_id] == sum(shifts.get((volunteer_id, d, s), 0)
                                                for d in all_days
                                                for s in all_shifts))

# Create a variable for the maximum number of shifts assigned to any volunteer
max_shifts = model.NewIntVar(0, SCHEDULING_HORIZON, "max_shifts")
# Iterate over volunteer IDs in total_shifts
model.AddMaxEquality(max_shifts, [total_shifts[volunteer_id] for volunteer_id in total_shifts])

# Create a variable for the minimum number of shifts assigned to any volunteer
min_shifts = model.NewIntVar(0, SCHEDULING_HORIZON, "min_shifts")
# Iterate over volunteer IDs in total_shifts
model.AddMinEquality(min_shifts, [total_shifts[volunteer_id] for volunteer_id in total_shifts])

In [ ]:
# Objective: Maximize fulfilled shift requests and minimize the difference between max and min shifts
# fairness_weight = 100  # Adjust this weight to balance between request fulfillment and fairness
# model.Maximize(
#     sum(shift_availability[n][d][s] * shifts[(n, d, s)]
#         for n in all_volunteers
#         for d in all_days
#         for s in all_shifts) -
#     fairness_weight * (max_shifts - min_shifts)
# )

# Objective: Maximize fulfilled shift requests and minimize the difference between max and min shifts
fairness_weight = 100  # Adjust this weight to balance between request fulfillment and fairness
model.Maximize(
    sum(shift_availability[volunteer_id][d][s] * shifts.get((volunteer_id, d, s), 0) # Use .get() and actual volunteer IDs
        for volunteer_id in shift_availability  # Iterate through volunteer IDs in shift_availability
        for d in all_days
        for s in all_shifts) -
    fairness_weight * (max_shifts - min_shifts)
)


In [ ]:
# Solve the model
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 600.0  # Set a time limit of 5 minutes
status = solver.Solve(model)

In [ ]:
# Print the results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution found!")
    # Print the schedule
    for d in all_days:
        print(f"Day {d}:")
        for s in all_shifts:
            print(f"  Shift {s}:", end=" ")
            for volunteer_id in shift_availability:
                if solver.Value(shifts.get((volunteer_id, d, s), 0)) == 1:
                    # Get volunteer info
                    # Use v.id to access the id attribute of the Volunteer object
                    volunteer_info = next((v for v in volunteers if v.id == volunteer_id), None) if isinstance(volunteers, list) else volunteers.get(volunteer_id)

                    if volunteer_info:
                        volunteer_name = volunteer_info.name # Use .name to access the name attribute
                        volunteer_pioneer_status = volunteer_info.pioneer_status# Use .sex to access the sex attribute
                        print(f"Volunteer {volunteer_id} ({volunteer_name}, Pioneer: {volunteer_pioneer_status})", end=" ")
                    else:
                        print(f"Volunteer {volunteer_id} (Unknown)", end=" ")  # Handle missing info
            print()

    # Print volunteer assignment summary
    print("\nVolunteer Assignment Summary:")
    for volunteer_id in shift_availability:
        total_shifts_for_volunteer = solver.Value(total_shifts[volunteer_id])
        # Get volunteer info
        # Use v.id to access the id attribute of the Volunteer object
        volunteer_info = next((v for v in volunteers if v.id == volunteer_id), None) if isinstance(volunteers, list) else volunteers.get(volunteer_id)

        if volunteer_info:
            volunteer_name = volunteer_info.name # Use .name to access the name attribute
            volunteer_pioneer_status = volunteer_info.pioneer_status # Use .sex to access the sex attribute
            print(f"Volunteer {volunteer_id} ({volunteer_name}, Pioneer: {volunteer_pioneer_status}): {total_shifts_for_volunteer} shifts")
        else:
            print(f"Volunteer {volunteer_id} (Unknown): {total_shifts_for_volunteer} shifts")  # Handle missing info

else:
    print("No solution found.")

No solution found.


In [ ]:
print("\nStatistics")
print(f"  - status    : {solver.StatusName(status)}")
print(f"  - conflicts : {solver.NumConflicts()}")
print(f"  - branches  : {solver.NumBranches()}")
print(f"  - wall time : {solver.WallTime()} seconds")


Statistics
  - status    : INFEASIBLE
  - conflicts : 0
  - branches  : 0
  - wall time : 0.030359156 seconds
